<a href="https://colab.research.google.com/github/AdityaAVG/CNN/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
#Image transformations  # This function basically converts image data into tensors that model can easily understand .
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  #0.5 means mean and std of 0.5
])

In [ ]:
trainset=datasets.MNIST('MNIST_data/',download=True,train=True,transform=transform) # transform is converting image into tensors .
testset=datasets.MNIST('MNIST_data/',download=True,train=False,transform=transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True) #Shuffle ensures that data is shuffled at every epoch.
testloader=torch.utils.data.DataLoader(testset,batch_size=64,shuffle=True)

In [ ]:
# Building  the neural network

class Net(torch.nn.Module):
    def __init__(self):
      super(Net,self).__init__()  #Inheriting the parent class and initializing it.
      self.conv1=torch.nn.Conv2d(1,32,3,1) #No. of input channels, output channels ,size of filter and stride
      self.conv2=torch.nn.Conv2d(32,64,3,1) #Because the last layers output is 32 its input will be 32
      self.dropout=torch.nn.Dropout(0.20)
      self.fc1=torch.nn.Linear(64*5*5,128) #128 is the output here and the first one is input
      self.fc2=torch.nn.Linear(128,10)#We are taking 10 as output because the dataset has images of numbers till 9

    def forward(self,x):  # self is calling our model   # There is a pattern to observe here .
      x=self.conv1(x)
      x=F.relu(x)
      x=F.max_pool2d(x,2) #pooling
      x=self.conv2(x)
      x=F.relu(x)
      x=F.max_pool2d(x,2)
      x=self.dropout(x)

      x=torch.flatten(x,1)
      x=self.fc1(x)
      x=F.relu(x)
      x=self.fc2(x)
      x=F.log_softmax(x,dim=1) #gives the probability
      output=x
      return output

In [ ]:
model=Net().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for epoch in range(5):
  running_loss=0.0
  for images,labels in trainloader:
    images,labels=images.to(device),labels.to(device)
    outputs=model(images) #forward pass
    loss=criterion(outputs,labels)

    optimizer.zero_grad()

    loss.backward()
    optimizer.step() #Updates the model para

    running_loss+=loss.item()

  print("Epoch {} - Training loss: {}".format(epoch+1,running_loss/len(trainloader)))

Epoch 1 - Training loss: 0.1561893588219926
Epoch 2 - Training loss: 0.05163179286137553
Epoch 3 - Training loss: 0.038025092220832175
Epoch 4 - Training loss: 0.028907999179472697
Epoch 5 - Training loss: 0.02465875660296905


In [ ]:
correct=0
total=0
with torch.no_grad():
  for images,labels in testloader:
    images,labels=images.to(device),labels.to(device)
    outputs=model(images)
    _,predicted=torch.max(outputs,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 99 %
